In [7]:
import pandas as pd

train_data, test_data = [], []
fold_alloc = pd.read_csv("data/Fold_alloc.csv")
baseline = pd.read_csv("data/clinical.csv")
train_test = pd.read_csv("data/train_tests.csv")

# split discharge_mRS into 3 classes
baseline["discharge_mRS"] = baseline.apply(lambda x: 2 if x["discharge_mRS"] >= 4 else 1 if x["discharge_mRS"] >= 2 else 0 , axis=1)

baseline.describe()



,SUBJECT_ID,Age,Sex,discharge_mRS,NIHSS
count,494.000000,494.000000,494.000000,494.000000,494.000000
mean,248.022267,70.447368,1.544534,1.471660,14.882591
std,143.559578,15.427126,0.498518,0.771058,6.594175
min,1.000000,18.000000,1.000000,0.000000,1.000000
25%,124.250000,60.250000,1.000000,1.000000,10.000000
50%,247.500000,72.000000,2.000000,2.000000,15.000000
75%,370.750000,82.750000,2.000000,2.000000,19.000000
max,499.000000,103.000000,2.000000,2.000000,33.000000


In [8]:
K_FOLDS = 5
kfolds = [[] for i in range(K_FOLDS)]

# put each subject into a fold
for fold in fold_alloc.iterrows():
    fold = fold[1]
    kfolds[int(fold["fold"])-1].append(int(fold["ID"]))

In [9]:
from feature_selection import mrmr


radiomics = pd.read_csv("data/radiomics.csv")
ids = radiomics["SUBJECT_ID"]
radiomics.drop(["SUBJECT_ID"], axis=1, inplace=True)

features = mrmr(radiomics, baseline["discharge_mRS"], 40)
# drop the features that are not in the top 50
filtered_radiomics = radiomics
filtered_radiomics["SUBJECT_ID"] = ids
filtered_radiomics["discharge_mRS"] = baseline["discharge_mRS"]
filtered_radiomics.describe()

,diagnostics_Mask_resegmented_Mean,diagnostics_Mask_resegmented_Minimum,diagnostics_Mask_resegmented_Maximum,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,wavelet_LLL_glszm_ZoneEntropy,wavelet_LLL_glszm_ZonePercentage,wavelet_LLL_glszm_ZoneVariance,wavelet_LLL_ngtdm_Busyness,wavelet_LLL_ngtdm_Coarseness,wavelet_LLL_ngtdm_Complexity,wavelet_LLL_ngtdm_Contrast,wavelet_LLL_ngtdm_Strength,SUBJECT_ID,discharge_mRS
count,494.000000,494.000000,494.000000,494.000000,494.000000,4.940000e+02,494.000000,494.000000,494.000000,494.000000,...,494.000000,494.000000,4.940000e+02,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000
mean,88.047886,70.268178,250.924959,84.960801,89.599176,4.675355e+09,2.045672,2.183854,171.893803,250.924959,...,7.867067,0.053728,9.737599e+05,8.666640,0.000020,48261.005641,0.007574,10.824963,247.500000,1.471660
std,14.302900,18.375383,52.100451,14.143774,14.734932,1.765559e+09,0.362189,0.755037,101.322522,52.100451,...,0.237572,0.025968,4.818854e+05,6.369810,0.000004,59410.072173,0.020384,7.740159,142.749781,0.771058
min,55.885256,1.095676,163.668542,49.310312,56.977947,1.650720e+09,1.375635,1.208663,6.032764,163.668542,...,7.144230,0.018583,1.197628e+03,0.337260,0.000010,5340.423890,0.000404,1.091797,1.000000,0.000000
25%,78.862399,58.135956,220.591912,75.563993,80.082812,3.472027e+09,1.828881,1.806848,105.070491,220.591912,...,7.712033,0.040593,6.619287e+05,3.001270,0.000017,23274.405380,0.002689,6.744206,124.250000,1.000000
50%,85.925285,71.915817,236.076273,82.953834,87.224938,4.315152e+09,1.935002,1.938946,150.009445,236.076273,...,7.848451,0.048579,9.362091e+05,6.479707,0.000019,30756.258985,0.004428,8.734108,247.500000,2.000000
75%,96.123950,81.283835,261.115139,92.775992,97.610558,5.469309e+09,2.113334,2.182182,212.698664,261.115139,...,8.001806,0.058804,1.228227e+06,14.016292,0.000021,45482.467185,0.007173,11.697604,370.750000,2.000000
max,164.168014,148.506755,480.894475,158.659234,167.428326,1.385705e+10,4.087254,7.282189,878.598527,480.894475,...,8.695453,0.347722,3.166296e+06,31.977069,0.000039,611106.405200,0.346157,71.856422,494.000000,2.000000


In [10]:
# split the data into train and test

k_train = []
k_test = []
for i in range(K_FOLDS):
    # filter by subject id
    train = filtered_radiomics.loc[filtered_radiomics["SUBJECT_ID"].isin(kfolds[i])]
    test = filtered_radiomics.loc[~filtered_radiomics["SUBJECT_ID"].isin(kfolds[i])]
    # add discharge_mRS to the train and test sets
    # train["discharge_mRS"] = baseline.loc[baseline["SUBJECT_ID"].isin(kfolds[i])]["discharge_mRS"]
    # test["discharge_mRS"] = baseline.loc[baseline["SUBJECT_ID"].isin(kfolds[i])]["discharge_mRS"]

    train.drop("SUBJECT_ID", axis=1, inplace=True)
    test.drop("SUBJECT_ID", axis=1, inplace=True)
    # add to ds
    k_train.append(train)
    k_test.append(test)

for train, test in zip(k_train, k_test):
    # print shape of each set
    print(f"Train: {train.shape}, Test: {test.shape}")

Train: (482, 1120), Test: (12, 1120)
Train: (482, 1120), Test: (12, 1120)
Train: (481, 1120), Test: (13, 1120)
Train: (482, 1120), Test: (12, 1120)
Train: (486, 1120), Test: (8, 1120)


In [11]:
import tensorflow_decision_forests as tfdf
import random

def train_model(train, test):
    m = tfdf.keras.RandomForestModel(num_trees=60, verbose=0, random_seed=random.randint(0, 1000))

    m.fit(train, verbose=0)
    m.compile(metrics=["accuracy"])
    # return test and self eval scores
    return (m.evaluate(test, return_dict=True, verbose=0)["accuracy"], m.make_inspector().evaluation())

test_res = []
self_res = []
for i in range(3):
    for train, test in zip(k_train, k_test):
        train = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="discharge_mRS")
        test = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="discharge_mRS")
        # append the accuracy to the results
        test, self = train_model(train, test)
        test_res.append(test)
        self_res.append(self.accuracy)

2023-04-03 12:15:47.543791: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_155' with dtype double and shape [482]
	 [[{{node Placeholder/_155}}]]
[INFO 23-04-03 12:15:56.5235 EDT kernel.cc:1242] Loading model from path /tmp/tmp2zzx7az3/model/ with prefix eec9deda3ede49c9
[INFO 23-04-03 12:15:56.6543 EDT decision_forest.cc:660] Model loaded with 60 root(s), 5588 node(s), and 1002 input feature(s).
[INFO 23-04-03 12:15:56.6543 EDT abstract_model.cc:1311] Engine "RandomForestGeneric" built
[INFO 23-04-03 12:15:56.6547 EDT kernel.cc:1074] Use fast generic engine
2023-04-03 12:15:56.713371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a v

In [ ]:
print(
f"""
Test Accuracy:
Average: {sum(test_res)/len(test_res)}
Max: {max(test_res)}
Min: {min(test_res)}

Self Evaluation:
Average: {sum(self_res)/len(self_res)}
Max: {max(self_res)}
Min: {min(self_res)}
""")


Test Accuracy:
Average: 0.6467948794364929
Max: 0.75
Min: 0.5

Self Evaluation:
Average: 0.6126402874366605
Max: 0.6337448559670782
Min: 0.58298755186722

